In [1]:
!ls /dev/vid*

/dev/video0


In [41]:
import jetson_inference
import jetson_utils
from jetson_utils import cudaCrop,cudaDeviceSynchronize,cudaAllocMapped, saveImage
import time
import datetime
import os

In [3]:
#Instanciate Detection Objects
#net = jetson_inference.detectNet("ssd-mobilenet-v2",threshold=0.5)
starttime = time.time()
video_net = jetson_inference.detectNet("coco-dog",['--log-level=error'],threshold=0.5)
print(f"Time spent to Load Video: {int(time.time()-starttime)} seconds")

Time spent to Load Video: 201 seconds


In [4]:
# Define the video Source.
video_camera = jetson_utils.videoSource("/dev/video0")

In [3]:
display = jetson_utils.videoOutput("file://dog_video.mp4")

In [5]:
#Instanciate Image Recognition Objects
starttime = time.time()
image_net = jetson_inference.imageNet('googlenet',['--log-level=error'])
print(f"Time spent to Create Network: {int(time.time()-starttime)} seconds")

Time spent to Create Network: 4 seconds


In [45]:
dir(detection)

['Area',
 'Bottom',
 'Center',
 'ClassID',
 'Confidence',
 'Contains',
 'Height',
 'Instance',
 'Left',
 'ROI',
 'Right',
 'Top',
 'Width',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [50]:
#Run the live stream
while True:
    img = video_camera.Capture()
    detections = video_net.Detect(img)
    #display.Render(img)
    #display.SetStatus("Object Detection | Network {:.0f} FPS".format(video_net.GetNetworkFPS())) 

    #Get ClassID for the objects
    for detection in detections:
        class_idx = detection.ClassID
        class_confidence = detection.Confidence
        class_desc = video_net.GetClassDesc(class_idx)
        #print(f"{class_desc} at {class_confidence*100}%")

        #React if ClassID is that of a cat
        if (class_desc =="dog") and (class_confidence >= .9):
            print("Dog in view.  Species being identified")
            #snapshot = jetson_utils.cudaAllocMapped(width=int(detection.Left)-int(detection.Right),height=int(detection.Top)-int(detection.Bottom),format=img.format)
            roi = (int(detection.Left), int(detection.cTop), int(detection.Right), int(detection.Bottom))
            snapshot = cudaAllocMapped(width=roi[2]-roi[0], height=roi[3]-roi[1], format=img.format)
            cudaCrop(img, snapshot, roi)
            cudaDeviceSynchronize()
            timestamp = datetime.datetime.now().strftime('%Y%m%d-%H%M%S-%f')
            saveImage("dog.jpg",snapshot)#os.path.join(f"{timestamp}-{class_idx}.jpg"), snapshot)
            del snapshot
            dog_img = jetson_utils.loadImage("dog.jpg")
            dog_idx, confidence = image_net.Classify(dog_img)
            #             img = jetson_utils.loadImage(opt.filename)
#             print(f"Time spent to Load Image: {int(time.time()-starttime)} seconds")
#             class_idx, confidence = net.Classify(img)

            dog_class_desc = image_net.GetClassDesc(dog_idx)
            print("image is recognized as '{:s}' (class #{:d}) with {:f}% confidence".format(dog_class_desc, dog_idx, confidence * 100))
            

Dog in view.  Species being identified
image is recognized as 'bell pepper' (class #945) with 94.628906% confidence
Dog in view.  Species being identified
image is recognized as 'tennis ball' (class #852) with 65.380859% confidence
Dog in view.  Species being identified
image is recognized as 'jellyfish' (class #107) with 31.250000% confidence
Dog in view.  Species being identified
image is recognized as 'tennis ball' (class #852) with 26.928711% confidence
Dog in view.  Species being identified
image is recognized as 'tennis ball' (class #852) with 66.699219% confidence
Dog in view.  Species being identified
image is recognized as 'tennis ball' (class #852) with 45.971680% confidence
Dog in view.  Species being identified
image is recognized as 'tennis ball' (class #852) with 56.689453% confidence
Dog in view.  Species being identified
image is recognized as 'tennis ball' (class #852) with 43.505859% confidence
Dog in view.  Species being identified
image is recognized as 'tennis ball

KeyboardInterrupt: 